In [ ]:
#!pip install unsloth

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

from datasets import Dataset, DatasetDict, concatenate_datasets, load_dataset

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.bfloat16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth



In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B Unsloth-SmolLM2-360M-Lora
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    trust_remote_code=True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass


In [ ]:

from datasets import load_dataset
# 2. Tapaco Veri Setini Yükleyin
dataset = load_dataset("turkish-nlp-suite/InstrucTurca", split="train", cache_dir="/media/hosman/Yedek/Datasets/")  # Tapaco veri setini yükle
dataset2 = load_dataset("Metin/WikiRAG-TR", split="train", cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"question": "Input", "answer": "Output"})
dataset3 = load_dataset("kayrab/patient-doctor-qa-tr-167732",  cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"question_content": "Input", "question_answer": "Output"})
dataset4 = Dataset.from_csv("BKÜ Sınav Analizi.csv")

In [ ]:
dataset4 = dataset4.filter(lambda x: x["GPT"]!=None and x["Soru"]!=None)
dataset4 = dataset4.filter(lambda x: len(x["GPT"])<max_seq_length)

In [ ]:
dataset4

In [ ]:
dataset4 = dataset4.rename_columns({"Soru": "Input", "GPT": "Output"})

In [ ]:
dataset3 = concatenate_datasets([dataset3["train"], dataset3["test"]])
dataset = concatenate_datasets([dataset.select(range(int(len(dataset)*0.30))), dataset2, dataset3.select(range(1000))]).remove_columns(['id', 'context', 'is_negative_response', 'number_of_articles', 'ctx_split_points', 'correct_intro_idx', 'doctor_title', 'doctor_speciality'])

In [ ]:
dataset = dataset4

In [ ]:
from datasets import Dataset
import numpy as np

# None içeren satırları temizleyen fonksiyon
def remove_none_rows(example):
    return example["Input"] is not None and example["Output"] is not None

# None değerleri içeren satırları filtrele
dataset = dataset.filter(remove_none_rows)

In [ ]:
from unsloth.chat_templates import get_chat_template

# Tokenizer'a ChatML formatını uygulama
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",  # Desteklenen formatlar: zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},  # ShareGPT uyumu
    map_eos_token=True,  # <|im_end|> -> </s>
)

# Soruları ChatGPT formatına dönüştüren fonksiyon (System Prompt dahil)
def questions2gptFormat(rows):
    conversations = []
    
    # Sistem promptunu ekleyelim
    system_prompt = [
        {"from": "system", "value": "Sen bir tarım uzmanısın. Kullanıcının zirai hastalıklar, bitki hastalıkları, ilaçlar ve tarımsal ekipmanlar hakkında sorularını detaylı ve teknik bir şekilde cevapla."}
    ]

    def format_row(row):
        # Kullanıcı ve asistan mesajlarını ekleme
        conversations.append(system_prompt + [{"from": "user", "value": row[0]}, {"from": "assistant", "value": row[1]}])
                
    for i in zip(rows["Input"], rows["Output"]):
        format_row(i)

    return {"conversations": conversations}

# Formatlama fonksiyonu
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}


In [ ]:
dataset = dataset.map(questions2gptFormat, batched = True, batch_size=10000).remove_columns(["Input","Output"])

In [ ]:
dataset

In [ ]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
dataset = dataset.shuffle(seed=42)
dataset = dataset.shuffle(seed=41)
dataset = dataset.shuffle(seed=40)
dataset = dataset.shuffle(seed=39)

In [ ]:
""" from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,) """

In [ ]:
dataset[5]["conversations"]

In [ ]:
print(dataset[5]["text"])

In [ ]:
unsloth_template = \
    "{{ bos_token }}"\
    "{{ 'You are a helpful assistant to the user\n' }}"\
    "{% for message in messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ '>>> User: ' + message['content'] + '\n' }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ '>>> Assistant: ' + message['content'] + eos_token + '\n' }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '>>> Assistant: ' }}"\
    "{% endif %}"
unsloth_eos_token = "eos_token"

if False:
    tokenizer = get_chat_template(
        tokenizer,
        chat_template = (unsloth_template, unsloth_eos_token,), # You must provide a template and EOS token
        mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
        map_eos_token = True, # Maps <|im_end|> to </s> instead
    )

In [ ]:
train_dataset, eval_dataset = dataset.train_test_split(test_size=0.2, seed=42).values()


In [ ]:
""" import wandb
from transformers import TrainerCallback
import torch
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel

class WandBQuestionCallback(TrainerCallback):
    def __init__(self, tokenizer, model, questions, log_interval=500,**kwargs):
        self.tokenizer = get_chat_template(
            tokenizer,
            chat_template="chatml",
            mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
            map_eos_token=True,
        )
        
        self.model = model

        self.device = "cuda"
        self.questions = questions  # List of question strings
        self.log_interval = log_interval
        
    def on_step_end(self, args, state, control, **kwargs):
        # if state.global_step % self.log_interval == 0:
        #     wandb.log({"step": state.global_step})
        #     self.log_model_responses()
        if True:
            wandb.log({"step": state.global_step})
            self.log_model_responses()

    def log_model_responses(self,**kwargs):

        

        responses = {}
        for question in self.questions:
            messages = [
                {"from": "human", "value": question},
            ]
            inputs = self.tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt"
            ).to(self.device)
            
            outputs = self.model.generate(input_ids=inputs, max_new_tokens=256, temperature=0.2, top_p=0.9, do_sample=True)
            response = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
            responses[question] = response
        
        wandb.log({"model_responses": responses})
 """

In [ ]:
""" # Örnek sorular
questions = [
    "433 * b - 7420490 = -7413995 denklemini çöz.",
    "Türkiye'nin başkenti neresidir?",
    "E=mc^2 denkleminin fiziksel anlamı nedir?",
    "Merhaba.Nasılsın?",
    "Merhaba, dün diş çekimi yapıldıktan sonra bu sabah aşırı kanama ile hekime başvurdum. Pihtinin oluştuğunu, ancak kanamanın durmadığı gerekçesiyle dikiş işlemi uyguladı. Bugün herhangi bir kanama veya ağrı yok, yalnız dikiş bölgesinde mukusa benzer bir doku oluştu. Tekrar gitmem gerekir mi?",
    "Merhaba, ben 18 yaşındayım, geçen yıl elimin üst kısmı yanmıştı, şimdi iyileşti ancak elimin üstünde yanık izi kaldı. Bu iz için herhangi bir ilaç veya farklı tedavi yöntemi var mıdır?"
    "Mulan filminin hikayesi hangi kaynaktan esinlenmiştir?",
    "Kartografya günümüzde nasıl teknolojilerden faydalanıyor?"

]

# Callback'i oluştur
wandb_callback = WandBQuestionCallback(tokenizer, model, questions) """

In [ ]:
import wandb

wandb.init(project="Basic LLM Train",  resume="allow", ) #id="a7zeymst",id="ecibz7e4" id="dbaxrwf4"
wandb.watch(model, log="all")


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    #data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 6,
    packing = False, # Can make training 5x faster for short sequences.
    #callbacks=[wandb_callback],
    args = TrainingArguments(
       
        gradient_accumulation_steps = 1,
        num_train_epochs=30,  
        per_device_train_batch_size=32,       # GPU başına batch boyutu
        per_device_eval_batch_size=32,       # GPU başına batch boyutu
        learning_rate = 0.0001,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 300,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "Unsloth-Mistral-7B-0.3-BKU-GPT",
        report_to="wandb",                    # WandB veya diğer araçlara raporlama yok
        save_total_limit=2,                  # Sadece son iki checkpoint'i sakla
        save_steps=300,
        warmup_steps=10000,           # İlk 1000 adımda LR'yi yavaş yavaş artır
        #run_name="ag4pxuzz"
    ),
)

In [ ]:
""" from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
) """

In [ ]:
""" tokenizer.decode(trainer.train_dataset[5]["input_ids"]) """

In [ ]:
""" 
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]]) 
"""

In [ ]:
wandb.watch(model, log="all")
trainer_stats = trainer.train()

In [ ]:
model.save_pretrained("Unsloth-Mistral-7B-0.3-BKU-GPT") # Local saving
tokenizer.save_pretrained("Unsloth-Mistral-7B-0.3-BKU-GPT")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

INFERENCE

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.bfloat16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth



In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Unsloth-Mistral-7B-0.3-BKU-GPT/checkpoint-4500",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    trust_remote_code=True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass


In [ ]:
from unsloth.chat_templates import get_chat_template

# Tokenizer'a chat şablonunu uygula
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",  # Desteklenen formatlar: zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping={
        "role": "from",
        "content": "value",
        "user": "human",
        "assistant": "gpt"
    },  # ShareGPT tarzı eşleme
    map_eos_token=True  # <|im_end|> ifadesini </s> ile eşle
)

# Modeli çıkarım (inference) için hazırla
FastLanguageModel.for_inference(model)

# Kullanıcı mesajları
messages = [
    {"from": "human", "value": "TarımZeka?"}
]

# Mesajları tokenize et ve modele uygun hale getir
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,  # Üretim için gerekli
    return_tensors="pt"
).to("cuda")

# Modelden yanıt üret
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=max_seq_length,
    use_cache=True
)

# Yanıtları çözümle
decoded_outputs = tokenizer.batch_decode(outputs)

# Çıktıyı formatlayan fonksiyon
def format_chat_output(decoded_outputs):
    formatted_text = decoded_outputs[0].replace("<|im_start|>user\n", "🗣 **Kullanıcı:**\n") \
                                      .replace("<|im_start|>assistant\n", "🤖 **Asistan:**\n") \
                                      .replace("<|im_end|>", "").strip()
    return formatted_text

# Formatlanmış çıktıyı ekrana yazdır
print(format_chat_output(decoded_outputs))


In [ ]:
""" import pandas as pd
import torch
import random
from unsloth.chat_templates import get_chat_template
from difflib import SequenceMatcher
from tqdm import tqdm

# GPU kullanımı kontrol et
device = "cuda" if torch.cuda.is_available() else "cpu"

# Chat şablonunu Tokenizer'a uygula
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",
    mapping={
        "role": "from",
        "content": "value",
        "user": "human",
        "assistant": "gpt"
    },
    map_eos_token=True
)

# Modeli çıkarım (inference) için hazırla
FastLanguageModel.for_inference(model)
model.to(device)  # GPU'ya taşı

# CSV dosyasını oku
csv_file = "BKÜ Sınav Analizi.csv"
df = pd.read_csv(csv_file)

# Veriyi karıştır ve sadece %25'ini kullan
df = df.sample(frac=1, random_state=42)  # Karıştır
df = df.sample(frac=0.99, random_state=42)  # %25'ini seç

# Doğru tahminleri saymak için sayaç
correct_count = 0
total_questions = len(df)

# Benzerlik hesaplayan fonksiyon
def similarity(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

# Soruları tek tek modele gönder ve doğruluğu ölç
for index, row in tqdm(df.iterrows(), total=len(df)):
    question = row["Soru"]
    correct_answer = row["GPT"]

    messages = [{"from": "human", "value": question}]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(device)

    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=512,
        use_cache=True
    )

    decoded_outputs = tokenizer.batch_decode(outputs)
    
    # Model çıktısını formatla
    model_answer = decoded_outputs[0].replace("<|im_start|>user\n", "").replace("<|im_start|>assistant\n", "").replace("<|im_end|>", "").strip()
    model_answer = model_answer[len(question):]

    # Benzerlik oranını hesapla
    match_ratio = similarity(str(model_answer), str(correct_answer))

    # %80'den büyükse doğru kabul et
    if match_ratio > 0.8:
        correct_count += 1

    #print(f"Soru: {question}")
    #print(f"Model Cevabı: {model_answer}")
    #print(f"Gerçek Cevap: {correct_answer}")
    #print(f"Benzerlik: %{match_ratio * 100:.2f}")
   # print("-" * 50)

# Doğruluk yüzdesini hesapla
accuracy = (correct_count / total_questions) * 100
print(f"Modelin doğruluk oranı: %{accuracy:.2f}")
 """